In [5]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import os  # For directory checks

# Set random seeds for reproducibility
np.random.seed(42)
random.seed(42)

# Generate 1000 companies
n_companies = 1000
companies = pd.DataFrame({
    'Company_id': range(1, n_companies + 1),
    'Onboarded_date': pd.date_range(start='2023-10-01', end='2025-09-30', periods=n_companies).date,
    'Onboarded_last6months': np.random.choice([True, False], n_companies, p=[0.2, 0.8])  # 20% recent
})

# Generate transactions
n_tx_per_company = 24  # Historical 24 months
transactions = []
launch_date = datetime(2025, 10, 1).date()
end_date = datetime(2025, 10, 17).date()  # Up to current date

for _, company in companies.iterrows():
    company_id = company['Company_id']
    onboarded_date = company['Onboarded_date']
    
    # Historical tx (up to Sep 30)
    for month in range(n_tx_per_company):
        tx_date = onboarded_date + timedelta(days=30*month)
        if tx_date > datetime(2025, 9, 30).date():
            continue
        is_third = np.random.choice([True, False], p=[0.3, 0.7])
        amount = np.random.lognormal(mean=10, sigma=1) * 1000
        fx_rev = amount * np.random.uniform(0.001, 0.005)
        if tx_date < launch_date:
            fee_rev = amount * 0.005
        else:
            fee_rev = amount * (0 if not is_third else 0.01)
        tx_id = random.randint(1, 999999)
        transactions.append({
            'Company_id': company_id,
            'Transaction_id': tx_id,
            'Type': 'outbound',
            'Is_third_party': is_third,
            'Transaction_date': tx_date,
            'Transaction_amount_usd': round(amount, 2),
            'Fx_revenue_usd': round(fx_rev, 2),
            'Fee_revenue_usd': round(fee_rev, 2)
        })
    
    # Post-launch tx: 50% of companies get 1 tx in Oct 1-17 (sparse, realistic)
    if np.random.random() < 0.5:  # 50% probability
        tx_date = launch_date + timedelta(days=np.random.randint(0, 17))  # Random day Oct 1-17
        is_third = np.random.choice([True, False], p=[0.3, 0.7])
        amount = np.random.lognormal(mean=10, sigma=1) * 1000
        fx_rev = amount * np.random.uniform(0.001, 0.005)
        fee_rev = amount * (0 if not is_third else 0.01)  # New fees only
        tx_id = random.randint(1, 999999)
        transactions.append({
            'Company_id': company_id,
            'Transaction_id': tx_id,
            'Type': 'outbound',
            'Is_third_party': is_third,
            'Transaction_date': tx_date,
            'Transaction_amount_usd': round(amount, 2),
            'Fx_revenue_usd': round(fx_rev, 2),
            'Fee_revenue_usd': round(fee_rev, 2)
        })

# Create DataFrame and save
df = pd.DataFrame(transactions)
SAVE_PATH = 'xerto_sample_data_updated.csv'
df.to_csv(SAVE_PATH, index=False)

# Verify
print("Current working directory:", os.getcwd())
print(f"CSV saved to: {os.path.abspath(SAVE_PATH)}")
print("File exists:", os.path.exists(SAVE_PATH))
if os.path.exists(SAVE_PATH):
    print("File size:", os.path.getsize(SAVE_PATH), "bytes (~880 KB)")

print("\nDate range:", df['Transaction_date'].min(), "to", df['Transaction_date'].max())
print("Post-launch tx count (Oct 1-17):", len(df[df['Transaction_date'] >= launch_date]))
print("\nFirst 5 rows:")
print(df.head())
print("\nSummary Statistics:")
print(df.describe())

Current working directory: C:\Users\kapsa\anaconda_projects\db
CSV saved to: C:\Users\kapsa\anaconda_projects\db\xerto_sample_data_updated.csv
File exists: True
File size: 892739 bytes (~880 KB)

Date range: 2023-10-01 to 2025-10-17
Post-launch tx count (Oct 1-17): 506

First 5 rows:
   Company_id  Transaction_id      Type  Is_third_party Transaction_date  \
0           1          670488  outbound            True       2023-10-01   
1           1          116740  outbound           False       2023-10-31   
2           1           26226  outbound           False       2023-11-30   
3           1          777573  outbound            True       2023-12-30   
4           1          288390  outbound           False       2024-01-29   

   Transaction_amount_usd  Fx_revenue_usd  Fee_revenue_usd  
0             63867677.18       250929.69        319338.39  
1             24824979.64        90242.11        124124.90  
2             13992205.33        41384.74         69961.03  
3             

In [8]:
from statsmodels.stats.power import TTestIndPower
import pandas as pd

# Step 1: Calculate effect size (uplift / std dev)
uplift_relative = 0.10  # 10% target
cv = 0.20  # Coefficient of variation (std / mean) from historical data
effect_size = uplift_relative / cv
print(f"Effect Size (d): {effect_size}")

# Step 2: Base case sample size (one-sided t-test)
analysis = TTestIndPower()
n_base = analysis.solve_power(effect_size, alpha=0.05, power=0.8, alternative='larger')
print(f"Base n per group: {n_base:.0f}")

# Step 3: Sensitivity scenarios
scenarios = [
    {"name": "Smaller Effect (5% uplift)", "d": 0.05 / 0.20, "power": 0.8, "alpha": 0.05, "alt": "larger"},
    {"name": "Larger Effect (15%)", "d": 0.15 / 0.20, "power": 0.8, "alpha": 0.05, "alt": "larger"},
    {"name": "Higher Power (90%)", "d": effect_size, "power": 0.9, "alpha": 0.05, "alt": "larger"},
    {"name": "Two-Sided (any direction)", "d": effect_size, "power": 0.8, "alpha": 0.05, "alt": "two-sided"},
    {"name": "Lower Variance (10% CV)", "d": 0.10 / 0.10, "power": 0.8, "alpha": 0.05, "alt": "larger"}
]

results = []
for scen in scenarios:
    if scen["alt"] == "two-sided":
        n = analysis.solve_power(scen["d"], alpha=scen["alpha"], power=scen["power"], alternative='two-sided')
    else:
        n = analysis.solve_power(scen["d"], alpha=scen["alpha"], power=scen["power"], alternative='larger')
    results.append({
        "Scenario": scen["name"],
        "Effect Size": round(scen["d"], 2),
        "Power": f"{scen['power']*100}%",
        "Alpha": scen["alpha"],
        "n Per Group": round(n)
    })

# Step 4: Display as table
df = pd.DataFrame(results)
print("\nSensitivity Analysis:")
print(df.to_string(index=False))

Effect Size (d): 0.5
Base n per group: 50

Sensitivity Analysis:
                  Scenario  Effect Size Power  Alpha  n Per Group
Smaller Effect (5% uplift)         0.25 80.0%   0.05          199
       Larger Effect (15%)         0.75 80.0%   0.05           23
        Higher Power (90%)         0.50 90.0%   0.05           69
 Two-Sided (any direction)         0.50 80.0%   0.05           64
   Lower Variance (10% CV)         1.00 80.0%   0.05           13


In [7]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# Set seed for reproducibility
np.random.seed(42)
random.seed(42)

# Generate data (as before)
n_companies = 1000
companies = pd.DataFrame({
    'Company_id': range(1, n_companies + 1),
    'Onboarded_date': pd.date_range(start='2023-10-01', end='2025-09-30', periods=n_companies).date,
    'Onboarded_last6months': np.random.choice([True, False], n_companies, p=[0.2, 0.8])
})

transactions = []
launch_date = datetime(2025, 10, 1).date()

for _, company in companies.iterrows():
    company_id = company['Company_id']
    onboarded_date = company['Onboarded_date']
    
    # Historical tx
    for month in range(24):
        tx_date = onboarded_date + timedelta(days=30*month)
        if tx_date > datetime(2025, 9, 30).date():
            continue
        is_third = np.random.choice([True, False], p=[0.3, 0.7])
        amount = np.random.lognormal(mean=10, sigma=1) * 1000
        fx_rev = amount * np.random.uniform(0.001, 0.005)
        fee_rev = amount * 0.005  # Pre-launch: always 0.5%
        tx_id = random.randint(1, 999999)
        transactions.append({
            'Company_id': company_id, 'Transaction_id': tx_id, 'Type': 'outbound',
            'Is_third_party': is_third, 'Transaction_date': tx_date,
            'Transaction_amount_usd': round(amount, 2), 'Fx_revenue_usd': round(fx_rev, 2),
            'Fee_revenue_usd': round(fee_rev, 2)
        })
    
    # Partial Oct tx (new fees as-generated)
    if np.random.random() < 0.5:
        tx_date = launch_date + timedelta(days=np.random.randint(0, 17))
        is_third = np.random.choice([True, False], p=[0.3, 0.7])
        amount = np.random.lognormal(mean=10, sigma=1) * 1000
        fx_rev = amount * np.random.uniform(0.001, 0.005)
        fee_rev = amount * (0 if not is_third else 0.01)
        tx_id = random.randint(1, 999999)
        transactions.append({
            'Company_id': company_id, 'Transaction_id': tx_id, 'Type': 'outbound',
            'Is_third_party': is_third, 'Transaction_date': tx_date,
            'Transaction_amount_usd': round(amount, 2), 'Fx_revenue_usd': round(fx_rev, 2),
            'Fee_revenue_usd': round(fee_rev, 2)
        })

df = pd.DataFrame(transactions)
df['Transaction_date'] = pd.to_datetime(df['Transaction_date'])
df['Month'] = df['Transaction_date'].dt.to_period('M')

# Assign groups
df['Group'] = np.where(df['Company_id'] % 2 == 0, 'Test', 'Control')

# Override Control Oct fees to 0.5%
df.loc[(df['Group'] == 'Control') & (df['Month'] == '2025-10'), 'Fee_revenue_usd'] = \
    df.loc[(df['Group'] == 'Control') & (df['Month'] == '2025-10'), 'Transaction_amount_usd'] * 0.005

# Aggregates
sep_data = df[df['Month'] == '2025-09']
oct_data = df[df['Month'] == '2025-10']

sep_rev = sep_data.groupby('Group')['Fee_revenue_usd'].sum().round(1)
oct_rev = oct_data.groupby('Group')['Fee_revenue_usd'].sum().round(1)

# Deltas
test_rev_delta = ((oct_rev['Test'] - sep_rev['Test']) / sep_rev['Test'] * 100).round(1)
control_rev_delta = ((oct_rev['Control'] - sep_rev['Control']) / sep_rev['Control'] * 100).round(1)

print("Sep Revenue (Control / Test):", sep_rev['Control'], "/", sep_rev['Test'])
print("Oct Revenue (Control / Test):", oct_rev['Control'], "/", oct_rev['Test'])
print("Control Rev Delta:", control_rev_delta)
print("Test Rev Delta:", test_rev_delta)

Sep Revenue (Control / Test): 87054631.9 / 93353287.1
Oct Revenue (Control / Test): 45334874.0 / 33797449.8
Control Rev Delta: -47.9
Test Rev Delta: -63.8


In [10]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# Set seed for reproducibility
np.random.seed(42)
random.seed(42)

# Generate data
n_companies = 1000
companies = pd.DataFrame({
    'Company_id': range(1, n_companies + 1),
    'Onboarded_date': pd.date_range(start='2023-10-01', end='2025-09-30', periods=n_companies).date,
    'Onboarded_last6months': np.random.choice([True, False], n_companies, p=[0.2, 0.8])
})

transactions = []
launch_date = datetime(2025, 10, 1).date()

for _, company in companies.iterrows():
    company_id = company['Company_id']
    onboarded_date = company['Onboarded_date']
    
    # Historical tx
    for month in range(24):
        tx_date = onboarded_date + timedelta(days=30*month)
        if tx_date > datetime(2025, 9, 30).date():
            continue
        is_third = np.random.choice([True, False], p=[0.3, 0.7])
        amount = np.random.lognormal(mean=10, sigma=1) * 1000
        fx_rev = amount * np.random.uniform(0.001, 0.005)
        fee_rev = amount * 0.005  # Pre-launch: always 0.5%
        tx_id = random.randint(1, 999999)
        transactions.append({
            'Company_id': company_id, 'Transaction_id': tx_id, 'Type': 'outbound',
            'Is_third_party': is_third, 'Transaction_date': tx_date,
            'Transaction_amount_usd': round(amount, 2), 'Fx_revenue_usd': round(fx_rev, 2),
            'Fee_revenue_usd': round(fee_rev, 2)
        })
    
    # Partial Oct tx (new fees as-generated)
    if np.random.random() < 0.5:
        tx_date = launch_date + timedelta(days=np.random.randint(0, 17))
        is_third = np.random.choice([True, False], p=[0.3, 0.7])
        amount = np.random.lognormal(mean=10, sigma=1) * 1000
        fx_rev = amount * np.random.uniform(0.001, 0.005)
        fee_rev = amount * (0 if not is_third else 0.01)
        tx_id = random.randint(1, 999999)
        transactions.append({
            'Company_id': company_id, 'Transaction_id': tx_id, 'Type': 'outbound',
            'Is_third_party': is_third, 'Transaction_date': tx_date,
            'Transaction_amount_usd': round(amount, 2), 'Fx_revenue_usd': round(fx_rev, 2),
            'Fee_revenue_usd': round(fee_rev, 2)
        })

df = pd.DataFrame(transactions)
df['Transaction_date'] = pd.to_datetime(df['Transaction_date'])
df['Month'] = df['Transaction_date'].dt.to_period('M')

# Assign groups
df['Group'] = np.where(df['Company_id'] % 2 == 0, 'Test', 'Control')

# Override Control Oct fees to 0.5% (for revenue consistency, though not needed here)
df.loc[(df['Group'] == 'Control') & (df['Month'] == '2025-10'), 'Fee_revenue_usd'] = \
    df.loc[(df['Group'] == 'Control') & (df['Month'] == '2025-10'), 'Transaction_amount_usd'] * 0.005

# Safe Month filtering (convert to string to avoid Period/string issues)
df['Month_str'] = df['Month'].astype(str)
sep_data = df[df['Month_str'] == '2025-09']
oct_data = df[df['Month_str'] == '2025-10']

# Volume (tx count)
sep_vol = sep_data.groupby('Group').size()
oct_vol = oct_data.groupby('Group').size()

test_vol_delta = ((oct_vol['Test'] - sep_vol['Test']) / sep_vol['Test'] * 100).round(1)
control_vol_delta = ((oct_vol['Control'] - sep_vol['Control']) / sep_vol['Control'] * 100).round(1)
vol_did = (test_vol_delta - control_vol_delta).round(1)

# Retention (unique active clients)
sep_active = sep_data.groupby('Group')['Company_id'].nunique()
oct_active = oct_data.groupby('Group')['Company_id'].nunique()

test_ret_delta = ((oct_active['Test'] - sep_active['Test']) / sep_active['Test'] * 100).round(1)
control_ret_delta = ((oct_active['Control'] - sep_active['Control']) / sep_active['Control'] * 100).round(1)
ret_did = (test_ret_delta - control_ret_delta).round(1)

# Prints
print("Sep Volume (Control / Test):", sep_vol['Control'], "/", sep_vol['Test'])
print("Oct Volume (Control / Test):", oct_vol['Control'], "/", oct_vol['Test'])
print("Control Vol Delta:", control_vol_delta)
print("Test Vol Delta:", test_vol_delta)
print("Volume DiD:", vol_did)

print("\nSep Active Clients (Control / Test):", sep_active['Control'], "/", sep_active['Test'])
print("Oct Active Clients (Control / Test):", oct_active['Control'], "/", oct_active['Test'])
print("Control Ret Delta:", control_ret_delta)
print("Test Ret Delta:", test_ret_delta)
print("Retention DiD:", ret_did)

Sep Volume (Control / Test): 492 / 492
Oct Volume (Control / Test): 258 / 248
Control Vol Delta: -47.6
Test Vol Delta: -49.6
Volume DiD: -2.0

Sep Active Clients (Control / Test): 492 / 492
Oct Active Clients (Control / Test): 258 / 248
Control Ret Delta: -47.6
Test Ret Delta: -49.6
Retention DiD: -2.0
